<img src="images/kiksmeisedwengougent.png" alt="Banner" width="1100"/>

<div>
    <font color=#690027 markdown="1">
        <h1>DETECTIE VAN HUIDMONDJES</h1> 
    </font>
</div>

<div class="alert alert-box alert-success">
In deze notebook verken je een diep neuraal netwerk om stomata op een microfoto van een bladafdruk te herkennen.
</div>

<div class="alert alert-block alert-warning"> 
Hoe een diep neuraal netwerk opgebouwd en getraind wordt, vind je terug in de notebook 'Fundamenten van een diep neuraal netwerk voor beeldherkenning'. 
</div>

Voer onderstaande code-cellen uit om de functies in deze notebook te kunnen gebruiken.

In [ ]:
import sys
!{sys.executable} -m pip install pymongo

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location(
name = "diep_neuraal_netwerk",                # geen verband tussen name en bestand, louter modulenaam
location = ".scripts/diep_neuraal_netwerk.py" # pad naar script
)
diep_neuraal_netwerk = importlib.util.module_from_spec(spec)
spec.loader.exec_module(diep_neuraal_netwerk)

<div>
    <font color=#690027 markdown="1">
        <h2>1. Het referentienetwerk</h2> 
    </font>
</div>

In deze notebook gebruik je een netwerk dat uit 3 convolutionele lagen met respectievelijk 32, 64 en 128 filters bestaat, aangevuld met 1 feedforward laag met 64 neuronen. Voor de training werd *stochastic gradient descent* als *optimizer* gebruikt, met een *learning rate* van 0,01. Er werd een beroep gedaan op dezelfde regularisatietechnieken als beschreven in de notebook 'Overfitting'.

De volgende afbeelding geeft de architectuur van dit netwerk weer (op dezelfde manier als de netwerken in de notebook 'Fundamenten van een diep neuraal netwerk voor beeldherkenning').

<img src="images/referentienetwerk.png" width="700"/>
<center> Figuur 1: Referentienetwerk.</center>

Om dit (getrainde) referentienetwerk in te laden voer je de volgende code-cel uit:

In [ ]:
diep_neuraal_netwerk.laad_referentie_model()

<div>
    <font color=#690027 markdown="1">
        <h2>2. Stomatadetectie op een microfoto van een blad</h2> 
    </font>
</div>

Het referentienetwerk kan een classificatie maken tussen 2 klassen ('Stoma' of 'Geen stoma') wanneer het een afbeelding van 120 pixels op 120 pixels krijgt als invoer. Het netwerk zou echter ook alle stomata op een grote afbeelding moeten kunnen vinden. Bijvoorbeeld de stomata op een foto van 1600 pixels op 1200 pixels, hierbij verondersteld dat de stomata ongeveer **120 op 120** pixels groot zijn op de foto. 

Om dit te verwezenlijken wordt een ***sliding window*** gebruikt. Dit is een (glijdend) venster van 120 pixels op 120 pixels waarmee een vierkantje uit de grote afbeelding wordt geknipt. Dat vierkantje kan als invoer van het getrainde netwerk worden gebruikt, waarop het netwerk dan een voorspelling zal geven voor dat vierkantje. Daarna schuift het venster naar rechts op met 10 pixels, en het proces herhaalt zich. Wanneer het vierkantje de volledige breedte van de afbeelding heeft doorlopen zal het opnieuw naar de linkerkant verspringen maar dan 10 pixels lager dan voorheen. Dit herhaalt zich tot de volledige afbeelding verwerkt werd met telkens sprongen van 10 pixels.

Volgende afbeelding geeft een illustratie. Rechts van de afbeelding zie je hoe zeker het netwerk is van de **voorspelling**. Een voorspelling van '1' betekent dat het netwerk 100 % zeker is dat het vierkantje een stoma bevat, een voorspelling van '0' betekent dat het netwerk oordeelt dat het vierkantje zeker geen stoma bevat. Zodra de waarde groter wordt dan 0,5 (dit is een opgegeven drempelwaarde: zie later), wordt het vierkantje geel.

<img src="images/slidingwindow.gif" width ="600"/><br>
<center> Figuur 2: Sliding window in actie.</center>

Voor elke vierkantje dat uit de grote afbeelding geknipt wordt, zal er op deze manier een voorspelling gemaakt worden.<br> Vaak zullen er meerdere vierkantjes rond een bepaalde stoma een positieve uitvoer, 'Stoma', geven omdat er slechts met 10 pixels wordt opgeschoven en de vierkantjes dus niet veel  van elkaar verschillen. Om deze vele positieve voorspellingen samen te voegen, gebruikt men clusteren. **Clustering** zal punten die dicht bij elkaar liggen als één cluster beschouwen en het middelpunt van deze clusters is dan de gevonden stoma. <br>

Volgende afbeelding geeft een voorbeeld van clustering waarbij de blauwe kruisjes de gevonden punten zijn en de rode kruisjes het middelpunt van elke cluster.

<img src="images/clustering.jpg" width="400"/>
<center> Figuur 3: Clustering.</center>

Bij de detectie speelt de **drempelwaarde** een belangrijke rol. Deze drempelwaarde bepaalt voor welke waarden van de uitvoer het netwerk de invoer als een stoma beschouwt. Als je bijvoorbeeld 0,5 als drempelwaarde neemt, dan zal alle uitvoer met een zekerheid groter dan 0,5 als 'Stoma' geclassificeerd worden en alle uitvoer kleiner dan 0,5 als 'Geen stoma'. 

Een belangrijke afweging die je moet maken bij het kiezen van de drempelwaarde is de verhouding tussen ***precision*** en ***recall***. 

-  Precision: het percentage van de gevonden stomata dat ook daadwerkelijk stomata zijn.</li>
-  Recall: het percentage van het aantal stomata op een afbeelding dat daadwerkelijk gevonden werd.</li>

Het is duidelijk dat een lage drempelwaarde zal zorgen voor een lage precision (er worden meer punten al stomata beschouwd, dus ook meer foute punten; er zijn dan meer vals positieven) maar een hoge recall (van de stomata gaan er ook meer gevonden worden). Andersom geldt dat een hoge drempelwaarde zal zorgen voor een hoge precision maar lage recall (er zullen meer vals negatieven zijn).

De volgende code-cel zal ervoor zorgen dat het referentienetwerk op drie afbeeldingen de stomata detecteert. Je kan de drempelwaarde (thr) aanpassen en het resultaat interpreteren.

In [ ]:
diep_neuraal_netwerk.vind_stomata()

<div>
    <font color=#690027 markdown="1">
        <h2>3. Adversarial learning</h2> 
     </font>
</div>

In het vakgebied van *adversarial learning* wordt er gezocht naar invoer die een machine kan **misleiden** en die dus een andere uitvoer oplevert dan de verwachte uitvoer. 

Je kan dit zelf toepassen op het referentienetwerk. Het is getraind om stomata te herkennen op een microfoto van een bladafdruk. Wanneer je een afbeelding invoert waarop iets volledig anders te zien is, zou je dus verwachten dat het netwerk geen enkele stoma vindt. Toch blijkt dit niet het geval.

Voer maar eens de volgende code-cel uit om te zien hoe het netwerk reageert op een afbeelding van een kat.

In [ ]:
diep_neuraal_netwerk.misleid_netwerk()

Je merkt dat het netwerk niet echt leert wat een stoma is maar gewoon patronen leert herkennen. De ogen van de kat hebben ongeveer dezelfde vorm als een stoma en worden daarom als stoma gedetecteerd.

Er bestaan tal van voorbeelden van dit soort 'adversarial afbeeldingen'. Binnen één gebied in adverserial learning gaat men op zoek naar afbeeldingen waarmee een (of meerdere) netwerken het moeilijk hebben. Een gekend voorbeeld zijn de afbeeldingen van chihuahua's en muffins waarbij een mens soms al twee keer moet kijken om het onderscheid te maken.

<img src="images/chihuahuamuffin.jpg" width="600"/>
<center> Figuur 4: Chihuahua en muffin.</center>

Een andere categorie van adverserial learning zijn de zichtbare manipulaties. In deze categorie is het onderscheid voor de mens duidelijk te zien, maar kan een machine wel misleid worden. Denk hierbij bijvoorbeeld aan een zelfrijdende auto die een bepaalde mens niet meer herkent omdat de mens bepaalde make-up heeft aangebracht. Onderzoekers zijn er ook in geslaagd om een netwerk voor gezichtsherkenning te misleiden door specifieke patronen op een brilmontuur te plaatsen. Zo slaagden ze erin het netwerk te laten uitvoeren dat het een bepaalde beroemdheid zag ([Accessorize to a Crime: Real and Stealthy Attacks onState-of-the-Art Face Recognition](https://www.cs.cmu.edu/~sbhagava/papers/face-rec-ccs16.pdf)). Volgende afbeelding toont bovenaan de testpersonen met de speciaal ontworpen bril, onderaan staan de personen die het netwerk weergeeft te zien.

<img src="images/adversarialglasses.jpg" width="600"/>
<center> Figuur 5: Misleiden met een bril.</center>

Een laatste categorie zijn de onzichtbare manipulaties. Het is mogelijk een afbeelding zo te manipuleren dat het voor de mens dezelfde afbeelding lijkt, maar dat een neuraal netwerk een totaal andere conclussie trekt. In een paper ([Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572)) hebben enkele onderzoekers een afbeelding van een panda aangepast door ruiswaarden toe te voegen. Een diep neuraal netwerk dat afbeeldingen classificeert, was voor de aanpassing 57,7 % zeker dat er een panda op de afbeelding stond; na de aanpassing oordeelde het netwerk met 99,3 % zekerheid dat het om een gibbon ging.

<img src="images/adversarialpanda.jpg" width="600"/>
<center> Figuur 3: Panda met ruis.</center>

Deze techniek kan ook misbruikt worden, denk maar aan een zelfrijdende auto die verkeersborden herkent met diepe neurale netwerken. Iemand met kwade bedoelingen zou een normaal stopbord kunnen vervangen door een aangepast stopbord dat er zo goed als hetzelfde uitziet voor de mens, maar niet kan gedetecteerd worden door de neurale netwerken van de auto. De auto zou in dat geval het stopbord negeren en levensgevaarlijk zijn voor de gebruiker.

### Opdracht 
- Ga op zoek naar een afbeelding waarmee je het netwerk in de notebook 'Stomatateller' zal proberen te misleiden. 
- Test uit. Gelukt?

<div>
    <h2>Met steun van</h2> 
</div>

<img src="images/kikssteun.png" alt="Banner" width="1100"/>

<img src="images/cclic.png" alt="Banner" align="left" width="100"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI Op School</a>, van F. wyffels, A. Meheus, T. Neutens & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 